<a href="https://colab.research.google.com/github/lcontrerasroa/macedonian/blob/main/notebooks/EAF2MAUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Fetch .eaf files directly from GitHub
import os
from pathlib import Path

# ton dépôt (change si besoin)
REPO_URL = "https://github.com/lcontrerasroa/macedonian.git"
TARGET_DIR = Path("/content/eaf_new")

if not TARGET_DIR.exists():
    os.system(f"git clone --depth 1 {REPO_URL} /content/macedonian")
    os.makedirs(TARGET_DIR, exist_ok=True)
    # copie uniquement les .eaf du dossier data/eaf_new dans eaf_new
    os.system("cp /content/macedonian/data/eaf_new/*.eaf /content/eaf_new/")
    print("✅ .eaf files imported from GitHub into /content/eaf_new")
else:
    print("Folder /content/eaf_new already exists — skipping clone.")

# vérification
!ls /content/eaf_new


Folder /content/eaf_new already exists — skipping clone.
new_only


In [6]:
# @title Inspect the internal tier structure of all EAF files
!pip -q install pympi-ling

from pathlib import Path
import pympi
import re

# === Paths ===
root = Path("/content/eaf_new")  # dossier contenant les *_new.eaf
assert root.exists(), "The folder /content/eaf_new doesn't exist. Upload or unzip your .eaf files first."

# === Function to summarize one EAF ===
def summarize_eaf(path):
    eaf = pympi.Elan.Eaf(str(path))
    tiers = eaf.get_tier_names()
    info = []
    for t in tiers:
        anns = eaf.get_annotation_data_for_tier(t)
        non_empty = [a for a in anns if a[2] and str(a[2]).strip()]
        info.append({
            "tier": t,
            "annotations": len(anns),
            "non_empty": len(non_empty)
        })
    return info

# === Loop over all .eaf files ===
results = []
for eaf_file in sorted(root.glob("*.eaf")):
    try:
        tiers = summarize_eaf(eaf_file)
        results.append((eaf_file.name, tiers))
    except Exception as e:
        results.append((eaf_file.name, f"ERROR: {e}"))

# === Display ===
for name, tiers in results:
    print(f"\n=== {name} ===")
    if isinstance(tiers, str):
        print("  ", tiers)
        continue
    for t in tiers:
        tname = t["tier"]
        nn = t["non_empty"]
        tot = t["annotations"]
        pct = (nn / tot * 100) if tot else 0
        print(f"  - {tname:30s} {nn:4d}/{tot:<4d} ({pct:5.1f}%) non-empty")



=== Macedonian_decata_so_zlatna_kosa_new.eaf ===
  - ref@SP1                         257/257  (100.0%) non-empty
  - rp@SP1                          109/109  (100.0%) non-empty
  - comm                              2/2    (100.0%) non-empty
  - qt@SP1                           61/61   (100.0%) non-empty
  - ft@SP1                          257/257  (100.0%) non-empty
  - lit@SP1                           0/0    (  0.0%) non-empty
  - tx@SP1                          257/257  (100.0%) non-empty
  - not@SP1                           4/4    (100.0%) non-empty
  - typ@SP1                          67/67   (100.0%) non-empty
  - mot@SP1                        1014/1014 (100.0%) non-empty
  - wps@SP1                         998/1014 ( 98.4%) non-empty
  - mb@SP1                         1319/1319 (100.0%) non-empty
  - ge@SP1                         1319/1319 (100.0%) non-empty
  - ps@SP1                         1319/1319 (100.0%) non-empty
  - par@SP1                          71/71   (100.0%) 

In [7]:
# @title Extract Cyrillic TXT per speaker from tx_cyr@SPx (fallback to tx@SPx)
!pip -q install pympi-ling

import re
from pathlib import Path
import pympi

SRC_DIR = Path("/content/eaf_new")           # tes .eaf
OUT_DIR = Path("/content/out_txt_cyr")       # txt cyrilliques
OUT_DIR.mkdir(parents=True, exist_ok=True)

tier_re_tx_cyr = re.compile(r"^tx_cyr@SP(\d+)$", re.I)
tier_re_tx     = re.compile(r"^tx@SP(\d+)$", re.I)

def get_speaker_tiers(eaf):
    names = eaf.get_tier_names()
    cyr = [(t, int(m.group(1))) for t in names if (m:=tier_re_tx_cyr.match(t))]
    lat = [(t, int(m.group(1))) for t in names if (m:=tier_re_tx.match(t))]
    # index par SP: préfère tx_cyr, sinon tx
    speakers = {}
    for t, sp in lat:
        speakers.setdefault(sp, ("tx@SP%d" % sp, t))
    for t, sp in cyr:
        speakers[sp] = ("tx_cyr@SP%d" % sp, t)
    # retourne liste (tiername_choisi, sp)
    chosen = []
    for sp, pair in sorted(speakers.items()):
        tag, tname = pair
        chosen.append((tname, sp, tag.startswith("tx_cyr")))
    return chosen

def concat_intervals(eaf, tiername):
    # annotations triées par début; ajoute un espace après chaque valeur
    ann = eaf.get_annotation_data_for_tier(tiername)
    ann = sorted(ann, key=lambda x: (x[0], x[1]))
    chunks = []
    for s,e,v in ann:
        if v and str(v).strip():
            chunks.append(str(v).strip() + " ")
    return "".join(chunks).strip()

count_files = 0
for eaf_path in sorted(SRC_DIR.glob("*.eaf")):
    eaf = pympi.Elan.Eaf(str(eaf_path))
    for tiername, sp, is_cyr in get_speaker_tiers(eaf):
        if not is_cyr:
            # si pas de tx_cyr pour ce SP, on prend tx@SPx tel quel
            pass
        txt = concat_intervals(eaf, tiername)
        if not txt:
            continue
        stem = eaf_path.stem.replace(".eaf","")
        out = OUT_DIR / f"{stem}_SP{sp}_cyr.txt"
        out.write_text(txt, encoding="utf-8")
        count_files += 1
        print(f"Wrote {out.name} ({len(txt)} chars)")
print(f"\n✅ Done. Wrote {count_files} TXT file(s) to {OUT_DIR}")


ValueError: too many values to unpack (expected 3)

In [ ]:
# @title Convert Cyrillic TXT → Spanish-like proxy (mk→es)
import re, unicodedata
from pathlib import Path

IN_DIR  = Path("/content/out_txt_cyr")
OUT_DIR = Path("/content/out_txt_proxy")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# --- cyr → latin de base (macédonien) ---
MK_CYR_TO_LAT = {
    "Ќ":"ḱ","ќ":"ḱ","Ѓ":"ǵ","ѓ":"ǵ","Ж":"ž","ж":"ž","З":"z","з":"z","Ѕ":"dz","ѕ":"dz",
    "Ч":"č","ч":"č","Џ":"dž","џ":"dž","Ш":"š","ш":"š","Ј":"j","ј":"j","Љ":"lj","љ":"lj",
    "Њ":"nj","њ":"nj","А":"a","а":"a","Б":"b","б":"b","В":"v","в":"v","Г":"g","г":"g",
    "Д":"d","д":"d","Е":"e","е":"e","И":"i","и":"i","К":"k","к":"k","Л":"l","л":"l",
    "М":"m","м":"m","Н":"n","н":"n","О":"o","о":"o","П":"p","п":"p","Р":"r","р":"r",
    "С":"s","с":"s","Т":"t","т":"t","У":"u","у":"u","Ф":"f","ф":"f","Х":"h","х":"h"
}
def mk_cyr_to_basic_latin(text:str)->str:
    return "".join(MK_CYR_TO_LAT.get(ch, ch) for ch in text)

# --- règles proxy espagnol (avec z→s et dz→ds comme demandé) ---
VOWELS = "aeiouáéíóúüAEIOUÁÉÍÓÚÜ"
def normalize_ws(t): return re.sub(r"\s+", " ", t).strip()
def strip_punct(t):
    t = re.sub(r"[„“”«»\"“”]", "", t)
    t = re.sub(r"[—–…]", " ", t)
    t = re.sub(r"[.,;:!?()\[\]{}/\\]", " ", t)
    return normalize_ws(t)

def apply_spanish_proxy_rules(text:str)->str:
    t = text
    # palatales
    t = t.replace("ḱ", "ky")
    t = t.replace("ǵ", "y")
    # affriquées et sibilantes
    t = t.replace("dž", "y")     # /dʒ/
    t = t.replace("dz", "ds")    # <- ta contrainte
    t = t.replace("č", "ch")
    t = t.replace("š", "s")
    t = t.replace("ž", "y")
    # lj, nj, j
    t = re.sub(r"\blj", "ll", t); t = t.replace("lj","ll")
    t = re.sub(r"\bnj", "ñ", t);  t = t.replace("nj","ñ")
    t = t.replace("j","y")
    # h (issu de х) → j dans environnements vocaliques
    t = re.sub(r"\b[hH](?=[%s])" % VOWELS, "j", t)
    t = re.sub(r"(?<=[%s])[hH](?=[%s])" % (VOWELS,VOWELS), "j", t)
    # k/g devant e,i → qu/gu
    t = re.sub(r"\bk(?=[eiéí])", "qu", t); t = re.sub(r"(?<=[^a-zA-Z])k(?=[eiéí])", "qu", t)
    t = re.sub(r"\bg(?=[eiéí])", "gu", t); t = re.sub(r"(?<=[^a-zA-Z])g(?=[eiéí])", "gu", t)
    # ailleurs, k → c (optionnel, plus “espagnol”)
    t = re.sub(r"\bk", "c", t);  t = re.sub(r"(?<=[^a-zA-Z])k", "c", t)
    return normalize_ws(t)

def mk_to_es_proxy(text:str)->str:
    t = strip_punct(text)
    t = mk_cyr_to_basic_latin(t)
    t = apply_spanish_proxy_rules(t)
    return t

written = 0
for f in sorted(IN_DIR.glob("*.txt")):
    raw = f.read_text(encoding="utf-8", errors="ignore")
    prox = mk_to_es_proxy(raw)
    out = OUT_DIR / f.name.replace("_cyr.txt", "_proxy_es.txt")
    out.write_text(prox + "\n", encoding="utf-8")
    written += 1
    print(f"Wrote {out.name} ({len(prox)} chars)")
print(f"\n✅ Done. Wrote {written} proxy TXT file(s) to {OUT_DIR}")
